In [ ]:
from pymongo import MongoClient
from tqdm import tqdm
import orjson
import os
import gc  # 가비지 컬렉션 모듈 추가


def put_match_data(collection_match, dump):
    collection_match.insert_many(dump)
    gc.collect()  # 데이터 삽입 후 가비지 컬렉션을 수동으로 실행


def collect_data(collection_match, json_data_gen):
    dump = []
    for data in json_data_gen:
        if verify_json(data):
            dump += data 

        if len(dump) >= 2000:  # 수정: 정확한 비교 연산자 사용
            put_match_data(collection_match, dump)
            dump = []  # dump 리스트를 초기화


def verify_json(json_data):
    return 'participants' in json_data[0]


def gen_json(file_gen):
    for file in tqdm(file_gen, mininterval=2):
        with open(file.path, 'rb') as f:
            try:
                json_data = orjson.loads(f.read())
            except Exception as e:
                print('Error reading {}: {}'.format(file.path, e))
                continue
        yield json_data


def migrate(stored_dir, collection_match):
    file_gen = os.scandir(stored_dir)
    json_data_gen = gen_json(file_gen)

    collect_data(collection_match, json_data_gen)

In [ ]:

if __name__ == '__main__':
    tier = 'gold'
    data_dir = '/home/data/lol/matches/gold/'

    client = MongoClient("mongodb://power16one5.iptime.org:27017/")
    db = client["loldb"]
    collection_match = db[f'{tier}_match']

    migrate(data_dir, collection_match)
